In [2]:
import numpy as np
import pandas as pd
import xlrd

import sys
import os
folder_name = '历史记录'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

import shutil
      
def copyDirectory(src, dest):
    try:
        shutil.copytree(src, dest)
    # Directories are the same
    except shutil.Error as e:
        print('Directory not copied. Error: %s' % e)
    # Any error saying that the directory doesn't exist
    except OSError as e:
        print('Directory not copied. Error: %s' % e)
        
# copy historical summary data
#month = int(sys.argv[1])
month = 1
cwd = os.getcwd()
print(cwd)

folder_list = ['成品销售明细表','成品仓库进出明细']
term_list = ['品牌']

if (month == 1):
    pass
else:
    for term in term_list:
        for folder_name in folder_list:
            for i in range(1, month):
                if (i != 11 and i != 10):
                    src= cwd[:-2] + '0' + str(i) + '/'+term+'/'+folder_name
                    dest = cwd + '/历史记录/'+term+'/'+folder_name + '_0' + str(i)
                else:
                    src = cwd[:-2] + str(i) + '/'+term+'/'+folder_name
                    dest = cwd + '/历史记录/'+term+'/'+folder_name + '_' + str(i)
                print(src)
                if not os.path.exists(dest):
                    copyDirectory(src, dest)

/mnt/d/mama文档/恒希2020/恒希2020_01


In [4]:
# add the historical cumulative selling information
# example

history_folder_flag = False
if os.path.exists('历史记录'):
    history_folder_flag = True

cwd= os.getcwd()
if history_folder_flag:
    # get previous months information
    key = '销售收入'
    df_months = list()
    for check_month in range(1,month):
        if check_month <10:
            previous_folder_name = '成品销售明细表_0'+ str(check_month)
        else:
            previous_folder_name = '成品销售明细表_'+ str(check_month)

        input_filename = '历史记录/品牌/'+previous_folder_name+'/summary_'+key+'_品牌分类.xlsx'
        df_check_month = pd.read_excel(input_filename, sheet_name = 0)

        # rebuild dataframe
        term_names = list(df_check_month.iloc[0,:].dropna())
        #category_names = list(df_check_month.iloc[1,:].dropna().unique())
        #print(category_names)
        category_names = ['金额', '装车金额', '合计金额']
        new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
        df_check_month= df_check_month.drop([0,1,2])
        df_check_month.set_index(df_check_month.columns.values[0], inplace=True)
        df_check_month.columns = new_columns
        
        df_months.append(df_check_month)
    
    # current month information
    folder_name = '成品销售明细表'
    input_filename = '品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_current_month = pd.read_excel(input_filename, sheet_name = 0)

    # rebuild dataframe
    term_names = list(df_current_month.iloc[0,:].dropna())
    category_names = ['金额', '装车金额', '合计金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_current_month= df_current_month.drop([0,1,2])
    df_current_month.set_index(df_current_month.columns.values[0], inplace=True)
    df_current_month.columns = new_columns
    
    df_months.append(df_current_month)

In [5]:
df_current_month = df_months[-1]
df_current_month

term           所有品牌                     维万家                     星晴         \
category         金额   装车金额     合计金额      金额   装车金额    合计金额      金额   装车金额   
销售收入 (01月)                                                                  
瓷片           566692  492.6   567185  124348  283.1  124631  125508  209.5   
销售费用        13426.7      0  13426.7       0      0       0       0      0   
浮雕              608      0      608       0      0       0       0      0   
圣陶居           241.2      0    241.2       0      0       0       0      0   
花片              160      0      160       0      0       0       0      0   
腰线              144      0      144       0      0       0       0      0   
总计           581272  492.6   581765  124348  283.1  124631  125508  209.5   

term                    出口   ...        金颐      佛山               恒希样板       \
category      合计金额      金额   ...      合计金额      金额 装车金额    合计金额    金额 装车金额   
销售收入 (01月)                   ...                                             
瓷片          125718  255929   ...     51495       0    0       0  29.4    0   
销售费用             0       0   ...         0       0    0       0     0    0   
浮雕               0       0   ...         0     608    0     608     0    0   
圣陶居              0       0   ...         0   241.2    0   241.2     0    0   
花片               0       0   ...         0     160    0     160     0    0   
腰线               0       0   ...         0       0    0       0     0    0   
总计          125718  255929   ...     51495  1009.2    0  1009.2  29.4    0   

term                  未注明                
category    合计金额       金额 装车金额     合计金额  
销售收入 (01月)                               
瓷片          29.4        0    0        0  
销售费用           0  13426.7    0  13426.7  
浮雕             0        0    0        0  
圣陶居            0        0    0        0  
花片             0        0    0        0  
腰线             0        0    0        0  
总计          29.4  13426.7    0  13426.7  

[7 rows x 27 columns]

In [6]:
#expenses_last_month = df_last_month.loc[:,(slice(None),'金额')]
#expenses_current_month = df_current_month.loc[:,(slice(None),'金额')]

#expenses_last_month = df_last_month.xs('合计金额', level='category', axis=1)
#expenses_current_month = df_current_month.xs('合计金额', level='category', axis=1)
#print(expenses_last_month.shape)
#print(expenses_current_month.shape)

expenses_months = list()
for i in range(0, len(df_months)):
    expenses_months.append( df_months[i].xs('合计金额', level='category', axis=1) )
    print(i+1, expenses_months[i].shape)
    

1 (7, 9)


In [7]:
#expenses of current month
expenses_months[-1]

term,所有品牌,维万家,星晴,出口,恒希,金颐,佛山,恒希样板,未注明
销售收入 (01月),,,,,,,,,
瓷片,567185,124631,125718,255929,9382.5,51495,0,29.4,0
销售费用,13426.7,0,0,0,0,0,0,0,13426.7
浮雕,608,0,0,0,0,0,608,0,0
圣陶居,241.2,0,0,0,0,0,241.2,0,0
花片,160,0,0,0,0,0,160,0,0
腰线,144,0,0,0,144,0,0,0,0
总计,581765,124631,125718,255929,9526.5,51495,1009.2,29.4,13426.7


In [8]:
expenses_cumulate = (pd.concat(expenses_months, sort=False)
                     .groupby(level=0)
                     .sum()
                     .sort_values(by='所有品牌', ascending =False)
                    )

expenses_cumulate

term,所有品牌,维万家,星晴,出口,恒希,金颐,佛山,恒希样板,未注明
销售收入 (01月),,,,,,,,,
总计,581764.76,124631.3,125717.9,255928.8,9526.5,51495,1009.2,29.4,13426.66
瓷片,567184.90,124631.3,125717.9,255928.8,9382.5,51495,0.0,29.4,0.00
销售费用,13426.66,0.0,0.0,0.0,0.0,0,0.0,0.0,13426.66
浮雕,608.00,0.0,0.0,0.0,0.0,0,608.0,0.0,0.00
圣陶居,241.20,0.0,0.0,0.0,0.0,0,241.2,0.0,0.00
花片,160.00,0.0,0.0,0.0,0.0,0,160.0,0.0,0.00
腰线,144.00,0.0,0.0,0.0,144.0,0,0.0,0.0,0.00


In [9]:
columns_names = list(expenses_cumulate)
new_columns = pd.MultiIndex.from_product([columns_names, ['3累计合计金额']], names=['term', 'category'])
expenses_cumulate.columns = new_columns
expenses_cumulate

term,所有品牌,维万家,星晴,出口,恒希,金颐,佛山,恒希样板,未注明
category,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额
销售收入 (01月),,,,,,,,,
总计,581764.76,124631.3,125717.9,255928.8,9526.5,51495,1009.2,29.4,13426.66
瓷片,567184.90,124631.3,125717.9,255928.8,9382.5,51495,0.0,29.4,0.00
销售费用,13426.66,0.0,0.0,0.0,0.0,0,0.0,0.0,13426.66
浮雕,608.00,0.0,0.0,0.0,0.0,0,608.0,0.0,0.00
圣陶居,241.20,0.0,0.0,0.0,0.0,0,241.2,0.0,0.00
花片,160.00,0.0,0.0,0.0,0.0,0,160.0,0.0,0.00
腰线,144.00,0.0,0.0,0.0,144.0,0,0.0,0.0,0.00


In [10]:
df_current_month.rename(index=str, columns={'金额': '0金额', '装车金额': '1装车金额', '合计金额':'2合计金额'}, inplace=True)

new_df = pd.concat([df_current_month, expenses_cumulate], axis=1, sort=True).fillna(0.00)
new_df.sort_index(axis=1, inplace=True)
new_df.rename(index=str, columns={'0金额': '金额', '1装车金额': '装车金额', '2合计金额':'合计金额',
                                  '3累计合计金额': '累计合计金额'}, inplace=True)
new_df = new_df.sort_values(by=('所有品牌','累计合计金额'),ascending = False)
new_df

term          佛山                             出口                           \
category      金额 装车金额    合计金额  累计合计金额        金额 装车金额      合计金额    累计合计金额   
总计        1009.2    0  1009.2  1009.2  255928.8    0  255928.8  255928.8   
瓷片           0.0    0     0.0     0.0  255928.8    0  255928.8  255928.8   
销售费用         0.0    0     0.0     0.0       0.0    0       0.0       0.0   
浮雕         608.0    0   608.0   608.0       0.0    0       0.0       0.0   
圣陶居        241.2    0   241.2   241.2       0.0    0       0.0       0.0   
花片         160.0    0   160.0   160.0       0.0    0       0.0       0.0   
腰线           0.0    0     0.0     0.0       0.0    0       0.0       0.0   

term          恒希       ...         未注明                 维万家                   \
category      金额 装车金额  ...        合计金额    累计合计金额        金额   装车金额      合计金额   
总计        9526.5    0  ...    13426.66  13426.66  124348.2  283.1  124631.3   
瓷片        9382.5    0  ...        0.00      0.00  124348.2  283.1  124631.3   
销售费用         0.0    0  ...    13426.66  13426.66       0.0    0.0       0.0   
浮雕           0.0    0  ...        0.00      0.00       0.0    0.0       0.0   
圣陶居          0.0    0  ...        0.00      0.00       0.0    0.0       0.0   
花片           0.0    0  ...        0.00      0.00       0.0    0.0       0.0   
腰线         144.0    0  ...        0.00      0.00       0.0    0.0       0.0   

term                   金颐                     
category    累计合计金额     金额 装车金额   合计金额 累计合计金额  
总计        124631.3  51495    0  51495  51495  
瓷片        124631.3  51495    0  51495  51495  
销售费用           0.0      0    0      0      0  
浮雕             0.0      0    0      0      0  
圣陶居            0.0      0    0      0      0  
花片             0.0      0    0      0      0  
腰线             0.0      0    0      0      0  

[7 rows x 36 columns]

In [11]:
N_col= new_df.columns.levels[0].size
new_columns = new_df.columns.levels[0].drop('所有品牌').insert(0,'所有品牌')
              
# put them at the end of columns
if '无' in new_columns:
    new_columns = new_columns.drop('无').insert(N_col,'无')
if '未注明' in new_columns:
    new_columns = new_columns.drop('未注明').insert(N_col,'未注明')
    
new_df = new_df.reindex(new_columns, level='term', axis=1)

N_row = new_df.shape[0]
new_rows = new_df.index.drop('总计').insert(N_row, '总计')
new_df = new_df.reindex(new_rows)

new_df = new_df.round(2)
new_df

term           所有品牌                                   佛山                       \
category         金额   装车金额       合计金额     累计合计金额      金额 装车金额    合计金额  累计合计金额   
瓷片        566692.30  492.6  567184.90  567184.90     0.0    0     0.0     0.0   
销售费用       13426.66    0.0   13426.66   13426.66     0.0    0     0.0     0.0   
浮雕           608.00    0.0     608.00     608.00   608.0    0   608.0   608.0   
圣陶居          241.20    0.0     241.20     241.20   241.2    0   241.2   241.2   
花片           160.00    0.0     160.00     160.00   160.0    0   160.0   160.0   
腰线           144.00    0.0     144.00     144.00     0.0    0     0.0     0.0   
总计        581272.16  492.6  581764.76  581764.76  1009.2    0  1009.2  1009.2   

term            出口         ...          维万家               金颐              \
category        金额 装车金额    ...         合计金额    累计合计金额     金额 装车金额   合计金额   
瓷片        255928.8    0    ...     124631.3  124631.3  51495    0  51495   
销售费用           0.0    0    ...          0.0       0.0      0    0      0   
浮雕             0.0    0    ...          0.0       0.0      0    0      0   
圣陶居            0.0    0    ...          0.0       0.0      0    0      0   
花片             0.0    0    ...          0.0       0.0      0    0      0   
腰线             0.0    0    ...          0.0       0.0      0    0      0   
总计        255928.8    0    ...     124631.3  124631.3  51495    0  51495   

term                  未注明                           
category 累计合计金额        金额 装车金额      合计金额    累计合计金额  
瓷片        51495      0.00    0      0.00      0.00  
销售费用          0  13426.66    0  13426.66  13426.66  
浮雕            0      0.00    0      0.00      0.00  
圣陶居           0      0.00    0      0.00      0.00  
花片            0      0.00    0      0.00      0.00  
腰线            0      0.00    0      0.00      0.00  
总计        51495  13426.66    0  13426.66  13426.66  

[7 rows x 36 columns]

In [12]:
def create_cumulative_info_factory(month, key):
     # get previous months information
    folder_name = '成品仓库进出明细'
    df_months = list()
    for check_month in range(1,month):
        if check_month <10:
            previous_folder_name = folder_name+'_0'+ str(check_month)
        else:
            previous_folder_name = folder_name+'_'+ str(check_month)

        input_filename = '历史记录/品牌/'+previous_folder_name+'/summary_'+key+'_品牌分类.xlsx'
        df_check_month = pd.read_excel(input_filename, sheet_name = 0)

        # rebuild dataframe
        term_names = list(df_check_month.iloc[0,:].dropna())
        #category_names = list(df_check_month.iloc[1,:].dropna().unique())
        #print(category_names)
        category_names = ['数量(片)','金额']
        new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
        df_check_month= df_check_month.drop([0,1,2])
        df_check_month.set_index(df_check_month.columns.values[0], inplace=True)
        df_check_month.columns = new_columns
        
        df_months.append(df_check_month)
    
    # current month information
   
    input_filename = '品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_current_month = pd.read_excel(input_filename, sheet_name = 0)

    # rebuild dataframe
    term_names = list(df_current_month.iloc[0,:].dropna())
    category_names = ['数量(片)','金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_current_month= df_current_month.drop([0,1,2])
    df_current_month.set_index(df_current_month.columns.values[0], inplace=True)
    df_current_month.columns = new_columns
    
    df_months.append(df_current_month)  
    
    expenses_months = list()
    for i in range(0, len(df_months)):
        expenses_months.append( df_months[i].xs('金额', level='category', axis=1) )
        #print(i+1, expenses_months[i].shape)
    
    expenses_cumulate = (pd.concat(expenses_months, sort=False)
                     .groupby(level=0)
                     .sum()
                     .sort_values(by='所有品牌', ascending =False)
                    )

    
    columns_names = list(expenses_cumulate)
    new_columns = pd.MultiIndex.from_product([columns_names, ['2累计金额']], names=['term', 'category'])
    expenses_cumulate.columns = new_columns
    
    df_current_month.rename(index=str, columns={'数量(片)':'0数量(片)', '金额': '1金额'}, inplace=True)

    new_df = pd.concat([df_current_month, expenses_cumulate], axis=1, sort=True).fillna(0.00)
    new_df.sort_index(axis=1, inplace=True)
    new_df.rename(index=str, columns={'0数量(片)':'数量(片)', '1金额': '金额',
                                      '2累计金额':'累计金额(1-'+str(month)+'月)'}, inplace=True)
    
    new_df = new_df.sort_values(by=('所有品牌','累计金额(1-'+str(month)+'月)'),ascending = False)
    
    N_col= new_df.columns.levels[0].size
    new_columns = new_df.columns.levels[0].drop('所有品牌').insert(0,'所有品牌')
    if '无' in new_columns:
        new_columns = new_columns.drop('无').insert(N_col,'无')
    new_df = new_df.reindex(new_columns, level='term', axis=1)

    N_row = new_df.shape[0]
    new_rows = new_df.index.drop('总计').insert(N_row, '总计')
    new_df = new_df.reindex(new_rows)

    new_df = new_df.round(2)
    return new_df



    

In [13]:
def create_cumulative_info_selling(month, key):
    folder_name = '成品销售明细表'
    # get previous months information
    df_months = list()
    for check_month in range(1,month):
        if check_month <10:
            previous_folder_name = folder_name + '_0'+ str(check_month)
        else:
            previous_folder_name = folder_name + '_'+ str(check_month)

        input_filename = '历史记录/品牌/'+previous_folder_name+'/summary_'+key+'_品牌分类.xlsx'
        df_check_month = pd.read_excel(input_filename, sheet_name = 0)

        # rebuild dataframe
        term_names = list(df_check_month.iloc[0,:].dropna())
        #category_names = list(df_check_month.iloc[1,:].dropna().unique())
        #print(category_names)
        category_names = ['金额', '装车金额', '合计金额']
        new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
        df_check_month= df_check_month.drop([0,1,2])
        df_check_month.set_index(df_check_month.columns.values[0], inplace=True)
        df_check_month.columns = new_columns
        
        df_months.append(df_check_month)
    
    # current month information
    input_filename = '品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_current_month = pd.read_excel(input_filename, sheet_name = 0)

    # rebuild dataframe
    term_names = list(df_current_month.iloc[0,:].dropna())
    category_names = ['金额', '装车金额', '合计金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_current_month= df_current_month.drop([0,1,2])
    df_current_month.set_index(df_current_month.columns.values[0], inplace=True)
    df_current_month.columns = new_columns
    
    df_months.append(df_current_month)
    
    #extract expenses
    expenses_months = list()
    for i in range(0, len(df_months)):
        expenses_months.append( df_months[i].xs('合计金额', level='category', axis=1) )
        #print(i+1, expenses_months[i].shape)

    expenses_cumulate = (pd.concat(expenses_months, sort=False)
                     .groupby(level=0)
                     .sum()
                     .sort_values(by='所有品牌', ascending =False)
                    )

    columns_names = list(expenses_cumulate)
    new_columns = pd.MultiIndex.from_product([columns_names, ['3累计合计金额']], names=['term', 'category'])
    expenses_cumulate.columns = new_columns
    
    df_current_month.rename(index=str, columns={'金额': '0金额', '装车金额': '1装车金额',
                                                '合计金额':'2合计金额'}, inplace=True)

    new_df = pd.concat([df_current_month, expenses_cumulate], axis=1, sort=True).fillna(0.00)
    new_df.sort_index(axis=1, inplace=True)
    new_df.rename(index=str, columns={'0金额': '金额', '1装车金额': '装车金额', '2合计金额':'合计金额',
                                      '3累计合计金额': '累计合计金额(1-'+str(month)+'月)'}, inplace=True)
    new_df = new_df.sort_values(by=('所有品牌','累计合计金额(1-'+str(month)+'月)'),ascending = False)
    
    N_col= new_df.columns.levels[0].size
    new_columns = new_df.columns.levels[0].drop('所有品牌').insert(0,'所有品牌')

    if '无' in new_columns:
        new_columns = new_columns.drop('无').insert(N_col,'无')
    if '未注明' in new_columns:
        new_columns = new_columns.drop('未注明').insert(N_col,'未注明')

    new_df = new_df.reindex(new_columns, level='term', axis=1)

    N_row = new_df.shape[0]
    new_rows = new_df.index.drop('总计').insert(N_row, '总计')
    new_df = new_df.reindex(new_rows)

    new_df = new_df.round(2)
    return new_df



In [14]:
def handle_worksheet_format(writer, df_temp, title):
    workbook = writer.book
    worksheet = writer.sheets['sheet1']    
    
    for idx, col in enumerate(df_temp):  # loop through all columns
        series = df_temp[col]
        max_len = max(( series.astype(str).map(len).max() ,  # len of largest item
                        max( [len(str(x)) for x in series.name])*2  # len of column name/header
                      )) + 2  # adding a little extra space

        data_fmt = workbook.add_format({'num_format': '0.00',
                                        'align': 'right'})    
        worksheet.set_column(idx+1, idx+1, max_len, data_fmt)  # set column width

    # set index format
    series = df_temp.index
    max_len_idx = max((
            series.astype(str).map(len).max() * 2,  # len of largest item
            len(str(series.name))  # len of column name/header
            ))+2
    index_fmt = workbook.add_format()
    index_fmt.set_align('right')
    worksheet.set_column(0,0, max_len_idx, index_fmt)
    
    # set title 
    header_format = workbook.add_format({'bold': True,
                                     'text_wrap': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC', 
                                     'border': 1})
    worksheet_cols = df_temp.shape[1]+1
    
    if (worksheet_cols <=25):
        range_str = 'A1:'+chr(ord('A')+worksheet_cols-1)+'1'
    else:
        last_col_str = chr(ord('A')+ worksheet_cols // 26 - 1) + chr(ord('A')+ (worksheet_cols-1) % 26) + '1'
        range_str = 'A1:'+last_col_str    
    worksheet.merge_range(range_str,title,header_format) 


In [15]:
def output_cumulative_info_dataframe(month, key, folder_name, out_path, title=None):
    if not os.path.exists(out_path + '/' + folder_name):
        os.makedirs(out_path + '/'+ folder_name)
        
    output_filename = out_path +'/'+ folder_name+'/统计_'+key+'_品牌分类.xlsx'      
    if title is None:
        month_str = ' ('+ str(month)+'月)'
        title=key + month_str
        
    if (folder_name == '成品仓库进出明细'):
        grouped =  create_cumulative_info_factory(month, key)
    elif (folder_name == '成品销售明细表'):
        grouped = create_cumulative_info_selling(month, key)
    else:
        print('Error for folder name')
        exit()
        
        
    # write to Excel file
    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workbook
    grouped.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook    
    
    
    # handle the spreadsheet format
    handle_worksheet_format(writer, grouped, title)       

    writer.save() #saves workbook to file in python file directory
    writer.close()
    
        

In [14]:
out_path = '累计金额信息'
folder_name = '成品仓库进出明细'

key_list = ['销售成本','免费样板','免费广告']
for key in key_list:
    output_cumulative_info_dataframe(month, key, folder_name, out_path)


In [15]:
out_path = '累计金额信息'
folder_name = '成品销售明细表'

key_list = ['销售收入', '挂账样板', '免费样板','免费广告','免费费用','破损赔偿','物资自用']
for key in key_list:
    output_cumulative_info_dataframe(month, key, folder_name, out_path)

